# Amplitude model with $LS$-couplings

```{autolink-concat}
```

In [ ]:
import sympy as sp
from IPython.display import display

from polarimetry.amplitude import simplify_latex_rendering
from polarimetry.io import display_latex
from polarimetry.lhcb import load_model_builder, load_model_parameters
from polarimetry.lhcb.particle import load_particles

simplify_latex_rendering()

In [ ]:
model_choice = "Alternative amplitude model obtained using LS couplings"
particles = load_particles("../../data/particle-definitions.yaml")
amplitude_builder = load_model_builder(
    model_file="../../data/model-definitions.yaml",
    particle_definitions=particles,
    model_id=model_choice,
)
model = amplitude_builder.formulate()

In [ ]:
def simplify_notation(expr):
    def substitute_node(node):
        if isinstance(node, sp.Indexed):
            if node.indices[2:] == (0, 0):
                return sp.Indexed(node.base, *node.indices[:2])
        return node

    for node in sp.preorder_traversal(expr):
        new_node = substitute_node(node)
        expr = expr.xreplace({node: new_node})
    return expr


display(simplify_notation(model.intensity.args[0].args[0].args[0].cleanup()))

In [ ]:
display_latex({simplify_notation(k): v for k, v in model.amplitudes.items()})

In [ ]:
imported_parameter_values = load_model_parameters(
    "../../data/model-definitions.yaml",
    amplitude_builder.decay,
    model_choice,
    particle_definitions=particles,
)
model_symbols = model.full_expression.free_symbols

non_existent = set(imported_parameter_values) - set(model_symbols)
error_message = "Imported symbols that don't exist in model:\n  "
error_message += "\n  ".join(map(str, sorted(non_existent, key=str)))
assert non_existent == set(), error_message

undefined = (
    set(model_symbols)
    - set(imported_parameter_values)
    - set(model.parameter_defaults)
    - set(model.variables)
    - set(sp.symbols("sigma1:4", nonnegative=True))
)
undefined = {
    s
    for s in undefined
    if not str(s).endswith("{decay}")
    if not str(s).endswith("production}")
}
error_message = "Symbols in model that don't have a definition:\n  "
error_message += "\n  ".join(map(str, sorted(undefined, key=str)))
assert undefined == set(), error_message
model.parameter_defaults.update(imported_parameter_values)

In [ ]:
production_couplings = {
    key: value
    for key, value in model.parameter_defaults.items()
    if isinstance(key, sp.Indexed)
    if "production" in str(key.base)
    if str(value) != "1"
}
display_latex(production_couplings)

:::{seealso}
See {ref}`amplitude-model:Resonances and LS-scheme` for the allowed $LS$-values.
:::